In [ ]:
!unzip /home/bizon/Downloads/training_data_7jul2019.zip


In [ ]:

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertModel
from pytorch_pretrained_bert import GPT2Model, GPT2DoubleHeadsModel, GPT2Tokenizer,GPT2LMHeadModel

from allennlp.commands.elmo import ElmoEmbedder

import torch
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
from torch.utils.data.distributed import DistributedSampler

import numpy as np
from scipy.spatial.distance import cosine

In [ ]:
import random
import pickle
seed = 0
training_file = 'training_data_merged_130k'
random.seed(seed)
thinning = 1
with open(training_file,'r') as infile:
    
    data = infile.read().rstrip()
    data = data.replace('$ISEP','').replace('$OSEP','').replace('<message boundary>','[[CLS]]<message boundary>')
    for _ in range(5):
        data = data.replace("  "," ")
    data = data.split('<thread boundary>')[1:]
    data = [d for d in data if random.random() < thinning]
    data = [[m for m in d.split('<message boundary>')[1:]] for d in data]
    
    data = [[message.split('<input-output boundary>') for message in thread] for thread in data]
    
    data = [[[list(filter(lambda a: a != '',section.split('\n'))) for section in message] for message in thread] for thread in data]
    data = [[[[list(filter(lambda a: a != '',sub.replace('NEWLINE NEWLINE','\n\n').replace('NEWLINE','\n').split(' '))) for sub in section] for section in message] for message in thread] for thread in data]
    data = [[[message[0][0],message[0][1],message[1][0]] for message in thread] for thread in data]
    
      

In [ ]:

elmo = ElmoEmbedder(options_file='~/DownloadedModels/Elmo/elmo_2x4096_512_2048cnn_2xhighway_5.5B_options.json',
                    weight_file='~/DownloadedModels/Elmo/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.hdf5',
                    cuda_device=1)

In [ ]:
data_with_embeddings = []
batch_size = 64
from tqdm import tqdm_notebook
for thread in tqdm_notebook(data):
    batched = []
    for ii in list(range(0,len(thread),batch_size)):
        thing = [message[0] for message in thread[ii:ii+batch_size]]
        batched += elmo.embed_batch([message[0] for message in thread[ii:ii+batch_size]])
       
    data_with_embeddings.append((thread,batched))

In [ ]:
pickle.dump(data_with_embeddings,open(f'elmo_embedded_{training_file}_{seed}_{thinning}.pkl','wb'))

In [ ]:
print(' '.join(data[-1][0][0]))

In [ ]:
t = 0
for d in data:
    t += len(d)
print(t)